# DuckDB Tutorial for Beginners (40 minutes)

- https://duckdb.org/docs/

- https://claude.ai/chat/2d11d485-b4bd-435a-976f-c25b4a03f0c0

## Part 1: Introduction (10 minutes)

### What is DuckDB?
DuckDB is an embedded analytical database engine, similar to SQLite but optimized for analytical queries (OLAP) rather than transactional workloads (OLTP). Think of it as "SQLite for analytics."

### Key Features and Advantages over Pandas
1. **Performance**: 
   - Executes queries much faster than Pandas, especially for large datasets
   - Efficient columnar storage and vectorized query execution
   - Better memory management - doesn't need to load entire dataset into RAM

2. **SQL-first Approach**:
   - Write familiar SQL queries instead of chaining Pandas operations
   - More readable and maintainable code
   - Easier transition for those with SQL background

3. **Integration**:
   - Seamless integration with Pandas (read/write DataFrames)
   - Direct reading of Parquet, CSV, JSON files
   - Can query data directly from files without loading into memory

4. **Scale**:
   - Handles larger-than-memory datasets efficiently
   - Parallel query execution
   - Better resource utilization

## Part 2: Setup (5 minutes)

### Creating a Conda Environment
```bash
# Create new environment with Python 3.11
conda create -n duckdb python=3.11

# Activate environment
conda activate duckdb

# Install required packages
pip install duckdb pandas jupyter notebook pyarrow
```

### Verifying Installation
```python
import duckdb
print(duckdb.__version__)
```

In [1]:
import duckdb
print(duckdb.__version__)

1.2.0


## Part 3: Hands-on Tutorial (20 minutes)

### demo-01

In [2]:
import duckdb
import pandas as pd

# Create a simple DataFrame
df = pd.DataFrame({
    'id': range(1, 6),
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'age': [25, 30, 35, 28, 22],
    'department': ['IT', 'HR', 'IT', 'Finance', 'HR']
})

In [3]:
df 

,id,name,age,department
0,1,Alice,25,IT
1,2,Bob,30,HR
2,3,Charlie,35,IT
3,4,David,28,Finance
4,5,Eve,22,HR


In [4]:
sql_1 = """
SELECT department, 
           COUNT(*) as count, 
           AVG(age) as avg_age
    FROM employees
    GROUP BY department
    ORDER BY count DESC
"""

In [5]:
# Create a DuckDB connection
con = duckdb.connect()

# Register DataFrame as a table
con.register('employees', df)

# Simple query
result = con.execute(sql_1).fetchdf()

print(result)

  department  count  avg_age
0         IT      2     30.0
1         HR      2     26.0
2    Finance      1     28.0


In [6]:
sql_1_1 = """
SELECT department, 
           COUNT(*) as count, 
           AVG(age) as avg_age
    FROM df  -- employees
    GROUP BY department
    ORDER BY count DESC
"""

In [7]:
result_2 = duckdb.sql(sql_1_1).df()
result_2

,department,count,avg_age
0,IT,2,30.0
1,HR,2,26.0
2,Finance,1,28.0


In [8]:
file_csv_emp = "employee.csv"

In [9]:
df.to_csv(file_csv_emp, index=False)

### demo-02 - Import CSV

In [10]:
df_emp = duckdb.read_csv(file_csv_emp)

In [11]:
df_emp

┌───────┬─────────┬───────┬────────────┐
│  id   │  name   │  age  │ department │
│ int64 │ varchar │ int64 │  varchar   │
├───────┼─────────┼───────┼────────────┤
│     1 │ Alice   │    25 │ IT         │
│     2 │ Bob     │    30 │ HR         │
│     3 │ Charlie │    35 │ IT         │
│     4 │ David   │    28 │ Finance    │
│     5 │ Eve     │    22 │ HR         │
└───────┴─────────┴───────┴────────────┘

### demo-04 - DuckDB beats Pandas

In [12]:
import duckdb
import pandas as pd
import time

In [13]:
def panda_way(df):
    
    # Method 1: Pandas
    start_time = time.time()
    try:
        # Create modulo column first
        df['id_mod'] = df['id'] % 1000
        result_pd = df.groupby('id_mod')['value'].mean()
        pd_time = time.time() - start_time
#         print(f"Pandas time: {pd_time:.2f} seconds")
        return 0, f"Pandas time: {pd_time:.2f} seconds"
    except MemoryError:
#         print("Pandas crashed - Out of memory!")
        return -1, "Pandas crashed - Out of memory!"

def duck_way(df):
    # Method 2: DuckDB    
    start_time = time.time()
    try:
        result_duck = duckdb.sql(f"""
            WITH d0 AS (
                SELECT *, (id - (id/1000)*1000) as id_mod FROM df
            )
            SELECT id_mod, avg(value) as avg_value
            FROM d0
            GROUP BY id_mod
        """).df()
        duck_time = time.time() - start_time
#         print(f"DuckDB time: {duck_time:.2f} seconds")
        return 0, f"DuckDB time: {duck_time:.2f} seconds"
    except Exception as e: 
        return -1, str(e)

In [14]:
# Generate a large dataset (adjust size based on your demo machine)
for n_rows in [10_000_000, 50_000_000, 100_000_000, 
               500_000_000, 
               # 1_000_000_000,
              ]:
    
    print(f"n_rows: {n_rows} ...")
    
    # prepare dataframe 
    df = pd.DataFrame({
        'id': range(n_rows),
        'value': range(n_rows)
    })

    # test duck
    ncode, msg = duck_way(df)
    print(ncode, msg)
    
    # test panda
    ncode, msg = panda_way(df)
    print(ncode, msg)

n_rows: 10000000 ...
0 DuckDB time: 0.03 seconds
0 Pandas time: 0.29 seconds
n_rows: 50000000 ...
0 DuckDB time: 0.13 seconds
0 Pandas time: 1.74 seconds
n_rows: 100000000 ...
0 DuckDB time: 0.26 seconds
0 Pandas time: 3.65 seconds
n_rows: 500000000 ...
0 DuckDB time: 2.01 seconds
0 Pandas time: 64.29 seconds


**Pandas - Out-of-Memory Error** 


- Running: 
![Duck Beats Panda](panda-vs-duck-01.png)

- Failing: 
![Out-of-Memory](panda-vs-duck-01-out-of-memory.png)

This is a fantastic demonstration of the performance differences between Pandas and DuckDB! Let me analyze the key insights from your results:

Dataset Size Progression:
- 10M rows: Pandas (0.28s) vs DuckDB (0.10s) - ~2.8x faster
- 50M rows: Pandas (1.44s) vs DuckDB (0.10s) - ~14.4x faster
- 100M rows: Pandas (3.80s) vs DuckDB (0.15s) - ~25.3x faster
- 500M rows: Pandas (76.95s) vs DuckDB (1.93s) - ~39.9x faster
- 1B rows: Pandas (MemoryError) vs DuckDB (would likely work)

Key Observations:
1. The performance gap widens dramatically as data size increases
2. Pandas hit a memory error trying to allocate 14.9 GB for an array with shape (2, 1000000000)
3. DuckDB maintains near-linear scaling with data size
4. Pandas performance degradation is super-linear with size

This clearly demonstrates:
- DuckDB's superior memory efficiency through out-of-core processing
- The limitations of Pandas' in-memory model
- Why DuckDB is better suited for large-scale data analysis

It's particularly interesting that DuckDB maintained sub-2-second performance even at 500M rows while Pandas took over a minute before failing completely at 1B rows. This is exactly the kind of real-world benchmark that helps people understand when to choose each tool.

## Part 3: Hands-on Tutorial (20 minutes)

### A. Data Import
```python
import duckdb
import pandas as pd

# Create a connection
con = duckdb.connect()

# CSV Import
con.sql("""
    CREATE TABLE users AS 
    SELECT * FROM read_csv_auto('users.csv')
""")

# Parquet Import
con.sql("""
    CREATE TABLE transactions AS 
    SELECT * FROM read_parquet('transactions.parquet')
""")

# JSON Import
con.sql("""
    CREATE TABLE events AS 
    SELECT * FROM read_json_auto('events.json')
""")

# From Pandas DataFrame
df = pd.read_csv('data.csv')
con.sql("SELECT * FROM df")  # Direct query on DataFrame
```

### B. SQL Operations on DataFrames
```python
# Basic queries
result = con.sql("""
    SELECT 
        user_id,
        COUNT(*) as transaction_count,
        SUM(amount) as total_spent
    FROM transactions
    GROUP BY user_id
    ORDER BY total_spent DESC
    LIMIT 5
""").df()

# Joins
result = con.sql("""
    SELECT 
        u.name,
        t.transaction_date,
        t.amount
    FROM users u
    JOIN transactions t ON u.id = t.user_id
    WHERE t.amount > 1000
""").df()

# Window Functions
result = con.sql("""
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY amount DESC) as rank
    FROM transactions
""").df()
```

### C. Large Dataset Analysis
```python
# Reading a large Parquet file (>RAM size)
con.sql("""
    SELECT 
        date_trunc('month', transaction_date) as month,
        COUNT(*) as transaction_count,
        SUM(amount) as total_amount,
        AVG(amount) as avg_amount
    FROM read_parquet('large_transactions.parquet')
    GROUP BY month
    ORDER BY month
""").df()

# Efficient joins with large datasets
con.sql("""
    SELECT 
        category,
        COUNT(DISTINCT user_id) as unique_users,
        SUM(amount) as total_spent
    FROM read_parquet('large_transactions.parquet') t
    JOIN read_parquet('large_users.parquet') u 
        ON t.user_id = u.id
    GROUP BY category
    HAVING total_spent > 1000000
""").df()
```

## Part 4: Resources (5 minutes)

### GitHub 
- DuckDB Main Repository: https://github.com/duckdb/duckdb
- [Awesome DuckDB](https://github.com/davidgasquez/awesome-duckdb)


### Blog Posts
- [DuckDB vs Pandas vs Polars](https://motherduck.com/blog/duckdb-versus-pandas-versus-polars/)
- [DuckDB for Data Engineers by 10x](https://www.datacamp.com/tutorial/duckdb-to-speed-up-data-pipelines)
- [Using DuckDB in Python: A Comprehensive Guide](https://medium.com/@anshubantra/using-duckdb-in-python-a-comprehensive-guide-d14bc0b06546)

### YouTube Videos
- [Deep dive into DuckDB](https://duckdb.org/media/deepdive-into-duckdb/)

### Documentation
- Official Documentation: https://duckdb.org/docs/
- SQL Reference: https://duckdb.org/docs/sql/introduction
- Python API: https://duckdb.org/docs/api/python/overview

### Books
- [DuckDB in Action](https://github.com/duckdb-in-action/examples)

